In [1]:
import torch
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2Tokenizer
import tqdm
from PIL import Image
import io

In [2]:
import os
import base64
import requests
import openai
openai.organization = "org-thCr4LFe4RGs65Rdh4YKMLC8"
# First save your openai gpt key to a txt file called gptkey.txt and restrict reading privileges
# then run the following in the terminal $ export OPENAI_API_KEY=$(cat /path/to/gptkey.txt)
# Make sure you do this in the same session as which you launch this notebook. You will probably
# need to restart this jupyter session in order to set the key.
openai.api_key = os.getenv("OPENAI_API_KEY")

## Completion Example

In [4]:
# Example completion request
#response = openai.Completion.create(
#  model="text-davinci-003",
#  prompt="Say this is a test",
#  max_tokens=7,
#  temperature=0
#)

# Example Response object
#{
#  "id": "cmpl-uqkvlQyYK7bGYrRHQ0eXlWi7",
#  "object": "text_completion",
#  "created": 1589478378,
#  "model": "text-davinci-003",
#  "choices": [
#    {
#      "text": "\n\nThis is indeed a test",
#      "index": 0,
#      "logprobs": null,
#      "finish_reason": "length"
#    }
#  ],
#  "usage": {
#    "prompt_tokens": 5,
#    "completion_tokens": 7,
#    "total_tokens": 12
#  }
#}

In [5]:
## Alternatively use the chat version
#response = openai.ChatCompletion.create(
#  model="gpt-3.5-turbo",
#  messages=[
#    {"role": "system", "content": "You are a helpful assistant."},
#    {"role": "user", "content": "Hello!"}
#  ]
#)
#
#print(response.choices[0].message)

## Example Response
#{
#  "id": "cmpl-uqkvlQyYK7bGYrRHQ0eXlWi7",
#  "object": "text_completion",
#  "created": 1589478378,
#  "model": "text-davinci-003",
#  "choices": [
#    {
#      "text": "\n\nThis is indeed a test",
#      "index": 0,
#      "logprobs": null,
#      "finish_reason": "length"
#    }
#  ],
#  "usage": {
#    "prompt_tokens": 5,
#    "completion_tokens": 7,
#    "total_tokens": 12
#  }
#}

In [47]:
# Function to encode the image
def encode_image(image):
    """
    Takes a uint8 ndarray or an image file path
    """
    if type(image)==type(np.zeros(1)):
        if image.dtype!="uint8":
            image = np.uint8(image*255)
        img = Image.fromarray(image)
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG')
        img_byte_arr = img_byte_arr.getvalue()
        return base64.b64encode(img_byte_arr).decode('utf-8')
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


In [51]:
async def async_request(payload, headers):
    return requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

async def image_request(image, text, max_tokens=300): 
    # Getting the base64 string
    base64_image = encode_image(image)
    
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {openai.api_key}"
    }
    
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": text,
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": max_tokens,
    }
    
    return await async_request(payload, headers)

In [52]:
resp = await image_request(image=image, text="Please describe this image.")

{'model': 'gpt-4-vision-preview', 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Please describe this image.'}, {'type': 'image_url', 'image_url': {'url': ''}}]}], 'max_tokens': 300}


In [49]:
## Example with images
#red = np.array([1,0.5,0.5])
#green = np.array([0.5,1,.5])
#blue = np.array([.5,.5,1])
#
## Path to your image
#image = np.zeros((30,30,3))
#image[5:10,5:10] = blue
#image[5:10,20:25] = blue
#image[20:25,5:25] = green
#image[17:23,1:5] = green
#image[17:23,25:-1] = green
#image = np.uint8(image*255)
#plt.imshow(image)
#plt.show()
#
## Getting the base64 string
#base64_image = encode_image(image)
#
#headers = {
#  "Content-Type": "application/json",
#  "Authorization": f"Bearer {openai.api_key}"
#}
#
#payload = {
#  "model": "gpt-4-vision-preview",
#  "messages": [
#    {
#      "role": "user",
#      "content": [
#        {
#          "type": "text",
#          "text": "Please describe this image with a single sentence."
#        },
#        {
#          "type": "image_url",
#          "image_url": {
#            "url": f"data:image/jpeg;base64,{base64_image}"
#          }
#        }
#      ]
#    }
#  ],
#  "max_tokens": 300
#}
#
#async def async_request(payload, headers):
#    return requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#
#resp = await async_request(payload, headers)
#print(resp.json())

{'id': 'chatcmpl-8NxCySm6gze0Yl59brf0bhNH5PQbE', 'object': 'chat.completion', 'created': 1700719832, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 271, 'completion_tokens': 48, 'total_tokens': 319}, 'choices': [{'message': {'role': 'assistant', 'content': "You've provided an image that appears to be very small and low in resolution, depicting a simplistic face with a square mouth and square eyes, set against a dark background, which looks like a stylized, pixelated emoticon or icon."}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]}


# Curate Data

In [4]:
root_name = "./data/data/"
dset_paths = [os.path.join(root_name, d) for d in os.listdir(root_name)]
for i,dset in enumerate(dset_paths):
    print(i, dset)

0 ./data/data/ex_1000.world.geometric_only.ppn_1.cls_2.tlf.ts_100.sl_15.simple.aws_50.V0
1 ./data/data/ex_100.world.active.all.all.ppn_1.cls_2.tlf.ts_100.sl_15.gpt.aws_50.V0
2 ./data/data/ex_1000.world.geometric_only.ppn_1.cls_2.tlf.ts_100.sl_15.gpt.aws_50.V0


In [5]:
dset_path = dset_paths[2]

In [6]:
fnames = sorted([os.path.join(dset_path, d) for d in os.listdir(dset_path)])
for i,fname in enumerate(fnames):
    print(i, fname)

0 ./data/data/ex_1000.world.geometric_only.ppn_1.cls_2.tlf.ts_100.sl_15.gpt.aws_50.V0/metadata.txt
1 ./data/data/ex_1000.world.geometric_only.ppn_1.cls_2.tlf.ts_100.sl_15.gpt.aws_50.V0/train_1_1.pth
2 ./data/data/ex_1000.world.geometric_only.ppn_1.cls_2.tlf.ts_100.sl_15.gpt.aws_50.V0/val.pth


In [7]:
!cat {fnames[0]}

mean ln(ref_obj attns): 0.79
mean ln(ref_obj attns) without null: 0.80
mean ln(all attns): 0.79
null mem samples: 103/1000 (10.3%)



In [8]:
data = torch.load(fnames[2])

In [9]:
print("N Samples:", len(data))
print("Sample Keys:", data[0].keys())
for k in data[0].keys():
    try:
        print(k, "Keys:", data[0][k].keys())
    except: pass

N Samples: 200
Sample Keys: dict_keys(['context', 'question', 'answer', 'answer_mask', 'mems', 'world_configs'])
context Keys: dict_keys(['t_hash', 't_word', 't_float', 'r_hash', 'r_word', 'r_float', 't_mask', 'r_mask', 't_attn', 'r_attn', 'text', 'text_mask'])
question Keys: dict_keys(['query_text', 'query_text_mask', 'query_tlf', 'query_slf', 'masks', 'tree_rels', 'clause_types'])
world_configs Keys: dict_keys(['SL'])


In [10]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer("Hello world")["input_ids"]

[15496, 995]

In [11]:
contexts = [
    tokenizer.decode(d["context"]["text"]) for d in data
]
questions = [
    tokenizer.decode(d["question"]["query_text"]) for d in data
]
answers =  [
    tokenizer.decode(d["answer"]).replace("<|endoftext|>", "") for d in data
]

In [12]:
contexts[0]

'fake_agent is at location (5,7,7). fake_agent has pitch -0.01 and yaw -0.01. fake_agent has property agent. fake_agent has property self. birdie is at location (9,7,9). birdie has pitch 0.00 and yaw 0.58. birdie has property pig. birdie has property mottled. skyler is at location (5,7,7). skyler has pitch 0.00 and yaw -1.25. skyler has property cow. skyler has property mottled. indy is at location (9,7,8). indy has pitch 0.00 and yaw 2.63. indy has property sheep. indy has property mottled. bella is at location (9,7,6). bella has pitch 0.00 and yaw 2.32. bella has property sheep. bella has property mottled. speaker is at location (7,7,9). speaker has pitch 0.00 and yaw -0.04. cross is at location (5,7,7). cross has property shiny. cross has property fuchsia. cross has property _in_inventory. ring is at location (8,7,5). ring has property _on_ground. ring has property matte. ring has property green. ring has property small. cross is at location (5,7,7). cross has property _on_ground. c

In [13]:
questions[0]

' which object is closest from square?'

In [14]:
answers[0]

'indy'

In [15]:
val_data = torch.load(fnames[2])

In [16]:
val_contexts = [
    tokenizer.decode(d["context"]["text"]) for d in val_data
]
val_questions = [
    tokenizer.decode(d["question"]["query_text"]) for d in val_data
]
val_answers = [
    tokenizer.decode(d["answer"]).replace("<|endoftext|>", "")  for d in val_data
]

# GPT Responses

In [17]:
example_q = val_contexts[0] + val_questions[0]
example_q 

'fake_agent is at location (5,7,7). fake_agent has pitch -0.01 and yaw -0.01. fake_agent has property agent. fake_agent has property self. birdie is at location (9,7,9). birdie has pitch 0.00 and yaw 0.58. birdie has property pig. birdie has property mottled. skyler is at location (5,7,7). skyler has pitch 0.00 and yaw -1.25. skyler has property cow. skyler has property mottled. indy is at location (9,7,8). indy has pitch 0.00 and yaw 2.63. indy has property sheep. indy has property mottled. bella is at location (9,7,6). bella has pitch 0.00 and yaw 2.32. bella has property sheep. bella has property mottled. speaker is at location (7,7,9). speaker has pitch 0.00 and yaw -0.04. cross is at location (5,7,7). cross has property shiny. cross has property fuchsia. cross has property _in_inventory. ring is at location (8,7,5). ring has property _on_ground. ring has property matte. ring has property green. ring has property small. cross is at location (5,7,7). cross has property _on_ground. c

In [18]:
example_ans = val_answers[0]
example_ans

'indy'

### Chat GPT

Here we will see how chatgpt responds.

In [19]:
persona_prompt = "You are a brilliant spatial thinker."
print("Persona Prompt:", persona_prompt)
#print()
#print()
#print("Demonstration:")
#print(demonstration)

Persona Prompt: You are a brilliant spatial thinker.


In [20]:
intro = "I am going to give you a series of questions that start with information "+\
                "about a spatial environment followed by a question."+\
                " Please answer the questions accurately, as a single sentence ending with a period.\n\nQuestion: " + example_q +\
                "\nAnswer: " + example_ans + "."
print("Intro:")
print(intro)

Intro:
I am going to give you a series of questions that start with information about a spatial environment followed by a question. Please answer the questions accurately, as a single sentence ending with a period.

Question: fake_agent is at location (5,7,7). fake_agent has pitch -0.01 and yaw -0.01. fake_agent has property agent. fake_agent has property self. birdie is at location (9,7,9). birdie has pitch 0.00 and yaw 0.58. birdie has property pig. birdie has property mottled. skyler is at location (5,7,7). skyler has pitch 0.00 and yaw -1.25. skyler has property cow. skyler has property mottled. indy is at location (9,7,8). indy has pitch 0.00 and yaw 2.63. indy has property sheep. indy has property mottled. bella is at location (9,7,6). bella has pitch 0.00 and yaw 2.32. bella has property sheep. bella has property mottled. speaker is at location (7,7,9). speaker has pitch 0.00 and yaw -0.04. cross is at location (5,7,7). cross has property shiny. cross has property fuchsia. cross

In [21]:
def get_query(context, question, intro):
    query = "\n\nQuestion: "+ context + question + "\nAnswer:"
    return intro + query

In [22]:
example_query = get_query(val_contexts[1], val_questions[1], intro)
print("Example:", example_query)

Example: I am going to give you a series of questions that start with information about a spatial environment followed by a question. Please answer the questions accurately, as a single sentence ending with a period.

Question: fake_agent is at location (5,7,7). fake_agent has pitch -0.01 and yaw -0.01. fake_agent has property agent. fake_agent has property self. birdie is at location (9,7,9). birdie has pitch 0.00 and yaw 0.58. birdie has property pig. birdie has property mottled. skyler is at location (5,7,7). skyler has pitch 0.00 and yaw -1.25. skyler has property cow. skyler has property mottled. indy is at location (9,7,8). indy has pitch 0.00 and yaw 2.63. indy has property sheep. indy has property mottled. bella is at location (9,7,6). bella has pitch 0.00 and yaw 2.32. bella has property sheep. bella has property mottled. speaker is at location (7,7,9). speaker has pitch 0.00 and yaw -0.04. cross is at location (5,7,7). cross has property shiny. cross has property fuchsia. cro

In [76]:
#response = openai.ChatCompletion.create(
#  model="gpt-3.5-turbo",
#  messages=[
#    {"role": "system", "content": persona_prompt },
#    {"role": "user", "content": example_query}
#  ]
#)

{
  "role": "assistant",
  "content": "inst_seg."
}


In [77]:
#response

#<OpenAIObject chat.completion id=chatcmpl-80TvkSqHrguC8yz7xGy8d8bzIgJeC at 0x7f24c287fa70> JSON: {
#  "id": "chatcmpl-80TvkSqHrguC8yz7xGy8d8bzIgJeC",
#  "object": "chat.completion",
#  "created": 1695125744,
#  "model": "gpt-3.5-turbo-0613",
#  "choices": [
#    {
#      "index": 0,
#      "message": {
#        "role": "assistant",
#        "content": "inst_seg."
#      },
#      "finish_reason": "stop"
#    }
#  ],
#  "usage": {
#    "prompt_tokens": 1740,
#    "completion_tokens": 3,
#    "total_tokens": 1743
#  }
#}

<OpenAIObject chat.completion id=chatcmpl-80TvkSqHrguC8yz7xGy8d8bzIgJeC at 0x7f24c287fa70> JSON: {
  "id": "chatcmpl-80TvkSqHrguC8yz7xGy8d8bzIgJeC",
  "object": "chat.completion",
  "created": 1695125744,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "inst_seg."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1740,
    "completion_tokens": 3,
    "total_tokens": 1743
  }
}

In [82]:
print("Correct Answer:", val_answers[1])
print("GPT Answer:", response["choices"][0]["message"]["content"])

Correct Answer: (8, 7, 8)
GPT Answer: inst_seg.


In [84]:
price_per_1k_prompt = 0.0015
price_per_1k_complet = 0.002
cost = response["usage"]["prompt_tokens"]/1000*price_per_1k_prompt + response["usage"]["completion_tokens"]/1000*price_per_1k_complet
print("API Price Per Request:", cost)
print("Experiment Cost:", cost*len(contexts))

API Price Per Request: 0.002616
Experiment Cost: 2.0928


### API Calls

In [23]:
def request_gpt(context, question, intro):
    query = get_query(context, question, intro)
    return openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": persona_prompt },
        {"role": "user", "content": query}
      ]
    )


In [25]:
responses = []
for i,(context, question) in tqdm.tqdm(enumerate(zip(contexts, questions))):
    try:
        r = request_gpt(context, question, intro)
        responses.append(r)
    except:
        print("Exception occurred at", i)
        assert False
    if i>3: break
print("Total Responses:", len(responses))
print()
responses[0]

4it [00:03,  1.26it/s]

Total Responses: 5



<OpenAIObject chat.completion id=chatcmpl-80VE7QRfUnfGLFCBBwR6wG8zVO9Hn at 0x7fbf16a91c10> JSON: {
  "id": "chatcmpl-80VE7QRfUnfGLFCBBwR6wG8zVO9Hn",
  "object": "chat.completion",
  "created": 1695130727,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "cross."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1701,
    "completion_tokens": 2,
    "total_tokens": 1703
  }
}

In [29]:
contexts[2]

'fake_agent is at location (8,7,5). fake_agent has pitch 0.01 and yaw 0.01. fake_agent has property agent. fake_agent has property self. aggie is at location (9,7,7). aggie has pitch 0.00 and yaw -1.52. aggie has property cow. aggie has property brown. beans is at location (5,7,9). beans has pitch 0.00 and yaw -1.12. beans has property cow. beans has property mottled. amy is at location (9,7,8). amy has pitch 0.00 and yaw 2.34. amy has property rabbit. amy has property gray. bishop is at location (7,7,8). bishop has pitch 0.00 and yaw -2.50. bishop has property rabbit. bishop has property mottled. speaker is at location (9,7,5). speaker has pitch 0.03 and yaw -0.05. ball is at location (9,7,5). ball has property fuzzy. ball has property _in_others_inventory. inst_seg is at location (8,0,10). inst_seg has property spherical_shell. inst_seg is at location (4,6,3). inst_seg has property arch. inst_seg is at location (12,9,10). inst_seg has property disk. inst_seg is at location (2,6,1). i

In [30]:
questions[2]

' what did you do?'

In [31]:
answers[2]

'dig'

In [27]:
results = {
  "id":[],
  "created":[],
  "model": [],
  "message": [],
  "role": [],
  "finish_reason": [],
  "prompt_tokens": [],
  "completion_tokens": [],
  "context": [],
  "question": [],
  "answer": [],
}
for r,context,question,answer in zip(responses, contexts, questions, answers):
    results["id"].append(r["id"])
    results["created"].append(r["created"])
    results["model"].append(r["model"])
    results["message"].append(r["choices"][0]["message"]["content"])
    results["role"].append(r["choices"][0]["message"]["role"])
    results["finish_reason"].append(r["choices"][0]["finish_reason"])
    results["prompt_tokens"].append(r["usage"]["prompt_tokens"])
    results["completion_tokens"].append(r["usage"]["completion_tokens"])
    results["context"].append(context)
    results["question"].append(question)
    results["answer"].append(answer)
df = pd.DataFrame(results)
df

,id,created,model,message,role,finish_reason,prompt_tokens,completion_tokens,context,question,answer
0,chatcmpl-80VE7QRfUnfGLFCBBwR6wG8zVO9Hn,1695130727,gpt-3.5-turbo-0613,cross.,assistant,stop,1701,2,"fake_agent is at location (5,7,7). fake_agent ...",which object is closest from square?,indy
1,chatcmpl-80VE7dvxWKv5oFJVYXtKbaeJG5rHu,1695130727,gpt-3.5-turbo-0613,"(4,7,7).",assistant,stop,1439,7,"fake_agent is at location (5,7,7). fake_agent ...",what is the location 1 steps to my left?,"(8, 7, 8)"
2,chatcmpl-80VE8dQYZxw0KPMaC5cYu8ZEqJQD2,1695130728,gpt-3.5-turbo-0613,I answered the previous question accurately.,assistant,stop,1509,7,"fake_agent is at location (8,7,5). fake_agent ...",what did you do?,dig
3,chatcmpl-80VE9Jo72PPqZCJyv97qojUTbG9Cx,1695130729,gpt-3.5-turbo-0613,savannah.,assistant,stop,1607,3,"fake_agent is at location (7,7,9). fake_agent ...",which object has the largest z?,fake_agent
4,chatcmpl-80VE9Xajveh4QT2hmJeuIF4sjtxym,1695130729,gpt-3.5-turbo-0613,I answered the question accurately.,assistant,stop,1543,6,"fake_agent is at location (6,7,9). fake_agent ...",what did you do?,dig
